In [8]:
import numpy as np
from abc import ABC, abstractmethod

In [9]:
class Perceptron(ABC):
    def __init__(self, num_features, lr, epochs):
        self.W = np.random.uniform(-.01,.01,num_features) # add one for bias
        self.bias = random.uniform(-.01,.01)
        self.epochs = epochs
        self.lr = lr
    
    def __repr__(self):
        return self.W
    
    @abstractmethod
    def train(self):
        return
    
    def test(self, test_data, test_labels):
        misclassifications = 0
            
        for i in range(train_data.shape[0]):
            example = train_data[i]
            actual_label = train_labels[i]
            predicted_label = self.W.T.dot(example) + self.bias
            
            if (actual_label * predicted_label) < 0:
                self.W = self.W + (self.lr * actual_label * example)
                self.bias = self.bias + (self.lr * actual_label)
                    
                misclassifications += 1
        
        accuracy = (num_examples - misclassifications)/num_examples
        print("accuracy on test set is", accuracy, ".")
        return accuracy

In [10]:
class SimplePerceptron(Perceptron):
    def __init__(self, num_features, lr=.01, epochs=10):
        super(Perceptron, self).__init__(num_features, lr, epochs)
        
    def train(self, train_data, train_labels):
        num_examples = train_data.shape[0]
        
        for epoch in range(self.epochs):
            misclassifications = 0
            
            for i in range(num_examples):
                example = train_data[i]
                actual_label = train_labels[i]
                predicted_label = self.W.T.dot(example) + self.bias
                
                if (actual_label * predicted_label) < 0:
                    self.W = self.W + (self.lr * actual_label * example)
                    self.bias = self.bias + (self.lr * actual_label)
                    
                    misclassifications += 1
            
            if misclassifications == 0:
                print("epoch", epoch, "completed with 100% accuracy.")
                return (self.W, self.bias)
            else:
                accuracy = (num_examples - misclassifications)/num_examples
                print("epoch", epoch, "complete with accuracy", accuracy, ".")
        
        return (self.W, self.bias)

In [11]:
class DynamicPerceptron(Perceptron):
    def __init__(self, num_features, lr=.01, epochs=10):
        super(Perceptron, self).__init__(num_features, lr, epochs)
        
    def train(self, train_data, train_labels):
        timestep = 0
        num_examples = train_data.shape[0]
        
        for epoch in range(self.epochs):
            misclassifications = 0
            
            for i in range(num_examples):
                example = train_data[i]
                actual_label = train_labels[i]
                predicted_label = self.W.T.dot(example) + self.bias
                
                if (actual_label * predicted_label) < 0:
                    adjusted_lr = self.lr/(1+timestep)
                    self.W = self.W + (adjusted_lr * actual_label * example)
                    self.bias = self.bias + (adjusted_lr * actual_label)
                    
                    misclassifications += 1
            
            if misclassifications == 0:
                print("epoch", epoch, "completed with 100% accuracy.")
                return (self.W, self.bias)
            else:
                timestep += 1 
                accuracy = (num_examples - misclassifications)/num_examples
                print("epoch", epoch, "complete with accuracy", accuracy, ".")
        
        return (self.W, self.bias)

In [12]:
class MarginPerceptron(Perceptron):
    def __init__(self, num_features, margin=.01, lr=.01,  epochs=10):
        super(Perceptron, self).__init__(num_features, lr, epochs)
        self.margin = margin
        
    def train(self, train_data, train_labels):
        timestep = 1
        num_examples = train_data.shape[0]
        
        for epoch in range(self.epochs):
            misclassifications = 0
            
            for i in range(num_examples):
                example = train_data[i]
                actual_label = train_labels[i]
                predicted_label = self.W.T.dot(example) + self.bias
                
                if (actual_label * predicted_label) < margin:
                    adjusted_lr = self.lr/timestep
                    self.W = self.W + (adjusted_lr * actual_label * example)
                    self.bias = self.bias + (adjusted_lr * actual_label)
                    
                    misclassifications += 1
            
            if misclassifications == 0:
                print("epoch", epoch, "completed with 100% accuracy.")
                return (self.W, self.bias)
            else:
                timestep += 1 
                accuracy = (num_examples - misclassifications)/num_examples
                print("epoch", epoch, "complete with accuracy", accuracy, ".")
                
        return (self.W, self.bias)

In [13]:
class AveragedPerceptron(Perceptron):
    def __init__(self, num_features, margin=.01, lr=.01,  epochs=10):
        super(Perceptron, self).__init__(num_features, lr, epochs)
        
    def train(self, train_data, train_labels):
        
        timestep = 1
        num_examples = train_data.shape[0]
        
        averagedW = self.W
        averagedBias = self.bias
        
        for epoch in range(self.epochs):
            misclassifications = 0
            
            for i in range(num_examples):
                example = train_data[i]
                actual_label = train_labels[i]
                predicted_label = self.W.T.dot(example) + self.bias
                
                if (actual_label * predicted_label) < 0:
                    self.W = self.W + (self.lr * actual_label * example)
                    self.bias = self.bias + (self.lr * actual_label)
                    
                    self.averagedW = averagedW + (self.lr * actual_label * example * timestep)
                    self.averagedBias  = averagedBias + (actual_label * timestep)
                    misclassifications += 1
            
            if misclassifications == 0:
                print("epoch", epoch, "completed with 100% accuracy.")
                
                self.W = self.W - ((1/timestep) * averagedW)
                self.bias = self.bias - ((1/timestep) * averagedBias)
                
                return (self.W, self.bias)
            else:
                timestep += 1 
                accuracy = (num_examples - misclassifications)/num_examples
                print("epoch", epoch, "complete with accuracy", accuracy, ".")
        
        self.W = self.W - ((1/timestep) * averagedW)
        self.bias = self.bias - ((1/timestep) * averagedBias)
        
        return (self.W, self.bias)
                           

In [14]:
class AggressiveMarginPerceptron(Perceptron):
    def __init__(self, num_features, margin=.01, lr=.01,  epochs=10):
        super(Perceptron, self).__init__(num_features, lr, epochs)
        self.margin = margin
        
    def train(self, train_data, train_labels):
        timestep = 1
        num_examples = train_data.shape[0]
        
        for epoch in range(self.epochs):
            misclassifications = 0
            
            for i in range(num_examples):
                example = train_data[i]
                actual_label = train_labels[i]
                predicted_label = self.W.T.dot(example) + self.bias
                
                if (actual_label * predicted_label) < margin:
                    adjusted_lr = (self.margin - (actual_label * predicted_label))/(predicted_label+1)
                    self.W = self.W + (adjusted_lr * actual_label * example)
                    self.bias = self.bias + (adjusted_lr * actual_label)
                    
                    misclassifications += 1
            
            if misclassifications == 0:
                print("epoch", epoch, "completed with 100% accuracy.")
                return (self.W, self.bias)
            else:
                timestep += 1 
                accuracy = (num_examples - misclassifications)/num_examples
                print("epoch", epoch, "complete with accuracy", accuracy, ".")
                
        return (self.W, self.bias)